<a href="https://colab.research.google.com/github/chewzzz1014/fyp/blob/master/ner/src/train_ner_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train NER Models

In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir spacy_ner_data

In [3]:
import json
import random
from sklearn.model_selection import train_test_split
import spacy
from spacy.tokens import DocBin

# Load JSON data
with open('/content/drive/MyDrive/FYP/Implementation/Resume Dataset/200_resumes_annotated.json', "r") as f:
    data = json.load(f)

def remove_overlapping_entities(entities):
    """Remove overlapping entities from the list."""
    entities = sorted(entities, key=lambda x: x[0])  # Sort by start position
    non_overlapping = []
    last_end = -1
    for start, end, label in entities:
        if start >= last_end:  # Only add if there's no overlap with the previous entity
            non_overlapping.append((start, end, label))
            last_end = end
    return non_overlapping

# Function to convert JSON data to Spacy's DocBin format
def convert_to_spacy_format(data):
    nlp = spacy.blank("en")  # Load a blank Spacy model
    doc_bin = DocBin()  # Container for our docs

    for item in data:
        text = item['data']['Text']  # Full document text
        entities = []

        for annotation in item['annotations'][0]['result']:
            start = annotation['value']['start']
            end = annotation['value']['end']
            label = annotation['value']['labels'][0]  # Entity label
            entities.append((start, end, label))

        entities = remove_overlapping_entities(entities)  # Remove overlapping entities
        # Create a Spacy doc and add entities to it
        doc = nlp.make_doc(text)
        spans = [doc.char_span(start, end, label=label) for start, end, label in entities]
        # Filter out None spans if Spacy can't align the character indices with tokens
        spans = [span for span in spans if span is not None]
        doc.ents = spans  # Assign entities to the doc
        doc_bin.add(doc)

    return doc_bin

# Split data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert train and test sets to Spacy format
train_doc_bin = convert_to_spacy_format(train_data)
test_doc_bin = convert_to_spacy_format(test_data)

# Save the train and test data to .spacy files
train_doc_bin.to_disk("spacy_ner_data/train_data.spacy")
test_doc_bin.to_disk("spacy_ner_data/test_data.spacy")

## Spacy NER

In [4]:
# create base_config.cfg and paste the config generated from spacy widget
# update train and test file path
!touch base_config.cfg

In [5]:
# generate config.cfg from base_config.cfg
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [6]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
# train model using hyperparameters set in config.cfg
# save trained model in spacy-output/ dir

# use cpu
# !python -m spacy train config.cfg --output ./spacy_output

# use gpu
!python -m spacy train config.cfg --gpu-id 0 --output ./spacy_output

!cp -r ./spacy_output /content/drive/MyDrive/FYP/Implementation/

ℹ Saving to output directory: spacy_output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.0001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    393.52    0.25    0.84    0.15    0.00
  3     500        318.16  39628.63   22.18   50.26   14.23    0.22
  6    1000        218.25  23196.31   37.41   32.47   44.11    0.37
  9    1500        125.29  21066.13   43.77   52.45   37.56    0.44
 12    2000        141.29  19069.86   47.18   61.24   38.38    0.47
 15    2500        144.90  16835.35   49.74   50.23   49.25    0.50
 18    3000        161.53  16130.90   49.34   53.33   45.90    0.49
 21    3500        167.70  14694.07   50.29   53.01   47.84    0.50
 25    4000        17

In [9]:
# evaluate trained model performance
# store output and visualization into result/ dir
!python -m spacy evaluate spacy_output/model-best spacy_ner_data/test_data.spacy -dp spacy_output

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     100.00
NER P   53.64 
NER R   48.88 
NER F   51.15 
SPEED   1175  


=============================== NER (per type) ===============================

                 P       R       F
NAME         97.06   89.19   92.96
LOC          48.57   58.62   53.12
PHONE       100.00   94.74   97.30
EMAIL        85.29   90.62   87.88
SKILL        32.52   26.28   29.07
WORK PER     84.71   94.33   89.26
JOB          66.06   45.86   54.14
STUDY PER    60.38   78.05   68.09
COMPANY      59.30   46.79   52.31
UNI          39.06   52.08   44.64
DEG          62.50   74.47   67.96

<IPython.core.display.HTML object>
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  F

In [10]:
# make prediction
import spacy
resume_text = '''
John Doe lives at 1234 Elm Street in Los Angeles, CA 90001. He can be reached at +1 (555) 123-4567 or via email at john.doe@example.com. John is a results-driven software engineer with over 5 years of experience in web development and cloud infrastructure, with strong knowledge of JavaScript, Python, and cloud technologies like AWS and Azure. Currently, he works as a Software Engineer at Google LLC in San Francisco, CA, where he has been employed since August 2019. In this role, he has developed scalable web applications using JavaScript, Node.js, and React, deployed and maintained cloud infrastructure on AWS, reducing downtime by 20%, and led a team of 4 engineers to enhance backend performance by 30%. Previously, he worked as a Junior Developer at Tech Innovators Inc. in Austin, TX, from July 2017 to July 2019, where he created RESTful APIs using Python and Flask, collaborated with front-end developers to build and deploy user-facing applications, and wrote unit and integration tests, improving code coverage by 15%.

John holds a Master of Science in Computer Science from the University of California, Berkeley, with a graduation date of May 2017, and a Bachelor of Science in Information Technology from the University of Texas at Austin, graduated in May 2015. His skillset includes proficiency in programming languages like Python, JavaScript, and Java; frameworks such as React, Flask, and Django; cloud platforms including AWS, Google Cloud, and Azure; as well as other tools like Git, Docker, Kubernetes, and SQL. He is certified as an AWS Certified Solutions Architect – Associate, earned in 2020, and as a Google Professional Cloud Architect, earned in 2021'
'''
nlp = spacy.load("spacy_output/model-best")
doc = nlp(resume_text.lower())

print(doc.ents)

for ent in doc.ents:
    print(f"{ent.text}: {ent.label_}")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


(john doe, at 1234, in los, (555) 123, john.doe@example.com, software engineer, software engineer, at google llc, at tech innovators inc, july 2017 to july 2019, master of science in computer science, from the university, berkeley, bachelor of science, the university, may 2015, python, associate,)
john doe: NAME
at 1234: PHONE
in los: LOC
(555) 123: PHONE
john.doe@example.com: WORK PER
software engineer: JOB
software engineer: JOB
at google llc: COMPANY
at tech innovators inc: COMPANY
july 2017 to july 2019: STUDY PER
master of science in computer science: DEG
from the university: UNI
berkeley: LOC
bachelor of science: DEG
the university: UNI
may 2015: STUDY PER
python: SKILL
associate,: UNI


In [11]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

## Flair NER

In [ ]:
!pip install flair

In [ ]:
# convert spacy data into flair data
import spacy
from spacy.tokens import DocBin
import os

def convert_spacy_to_flair(input_file, output_file):
    """
    Convert SpaCy binary format to Flair's CoNLL format.

    Args:
        input_file (str): Path to SpaCy binary file (.spacy)
        output_file (str): Path to output file for Flair format
    """
    # Load spaCy model
    nlp = spacy.blank("en")

    # Load the DocBin
    doc_bin = DocBin().from_disk(input_file)
    docs = list(doc_bin.get_docs(nlp.vocab))

    with open(output_file, 'w', encoding='utf-8') as f:
        for doc in docs:
            tokens = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]

            # Write tokens in CoNLL format
            for token in tokens:
                text, iob, ent_type = token

                # Convert spaCy IOB to CoNLL format
                if iob == 'O':
                    tag = 'O'
                else:
                    tag = f'{iob}-{ent_type}' if ent_type else 'O'

                # Write line: token and NER tag
                f.write(f'{text} {tag}\n')

            # Empty line between sentences
            f.write('\n')

def convert_spacy_json_to_flair(input_file, output_file):
    """
    Convert SpaCy JSON format to Flair's CoNLL format.

    Args:
        input_file (str): Path to JSON file with SpaCy annotations
        output_file (str): Path to output file for Flair format
    """
    import json

    nlp = spacy.blank("en")

    with open(input_file, 'r', encoding='utf-8') as f:
        training_data = json.load(f)

    with open(output_file, 'w', encoding='utf-8') as f:
        for example in training_data:
            text = example['text']
            ents = example.get('entities', [])

            # Create a spaCy doc
            doc = nlp(text)

            # Add entities to doc
            spans = []
            for start, end, label in ents:
                span = doc.char_span(start, end, label=label)
                if span is not None:
                    spans.append(span)
            doc.ents = spans

            # Convert to CoNLL format
            tokens = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]

            for token in tokens:
                text, iob, ent_type = token
                if iob == 'O':
                    tag = 'O'
                else:
                    tag = f'{iob}-{ent_type}' if ent_type else 'O'
                f.write(f'{text} {tag}\n')

            f.write('\n')

# Example usage for JSON format
flair_train_json = "flair_train.txt"
flair_test_json = "flair_test.txt"

convert_spacy_to_flair('/content/spacy_ner_data/train_data.spacy', flair_train_json)
convert_spacy_to_flair('/content/spacy_ner_data/test_data.spacy', flair_test_json)

FileNotFoundError: [Errno 2] No such file or directory: '/content/spacy_ner_data/train_data.spacy'

In [ ]:
# convert spacy data into flair data
import spacy
from spacy.training import Corpus

!python -m spacy download de_core_news_sm
nlp = spacy.load("de_core_news_sm")
corpus = Corpus("/content/spacy_ner_data/test_data.spacy")

data = corpus(nlp)

# Flair supports BIO and BIOES, see https://github.com/flairNLP/flair/issues/875
def rename_biluo_to_bioes(old_tag):
    new_tag = ""
    try:
        if old_tag.startswith("L"):
            new_tag = "E" + old_tag[1:]
        elif old_tag.startswith("U"):
            new_tag = "S" + old_tag[1:]
        else:
            new_tag = old_tag
    except:
        pass
    return new_tag


def generate_corpus():
    corpus = []
    n_ex = 0
    for example in data:
        n_ex += 1
        text = example.text
        doc = nlp(text)
        tags = example.get_aligned_ner()
        # Check if it's an empty list of NER tags.
        if None in tags:
            pass
        else:
            new_tags = [rename_biluo_to_bioes(tag) for tag in tags]
            for token, tag in zip(doc,new_tags):
                row = token.text +' '+ token.pos_ +' ' +tag + '\n'
                corpus.append(row)
            corpus.append('\n')
    return corpus

def write_file(filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        corpus = generate_corpus()
        f.writelines(corpus)

def main():
    write_file('flair_test.txt')

main()

  Using cached https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.7.0/de_core_news_sm-3.7.0-py3-none-any.whl (14.6 MB)
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# convert json into flair data
import json
import random
from typing import List, Dict, Tuple
import spacy
from collections import defaultdict

class NERConverter:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")

    def get_bioes_label(self, token_index: int, entity_length: int, current_position: int, label: str) -> str:
        """
        Convert to BIOES format
        - S-: Single token entity
        - B-: Beginning of multi-token entity
        - I-: Inside of multi-token entity
        - E-: End of multi-token entity
        - O: Outside
        """
        if entity_length == 1:
            return f'S-{label}'
        if current_position == 0:
            return f'B-{label}'
        if current_position == entity_length - 1:
            return f'E-{label}'
        return f'I-{label}'

    def convert_to_bioes_format(self, json_data: List[dict]) -> List[List[Tuple[str, str]]]:
        """Convert JSON annotations to BIOES format."""
        all_sentences = []

        for item in json_data:
            text = item['data']['Text']
            doc = self.nlp(text)

            # Initialize character-level labels
            char_labels = ['O'] * len(text)

            # First pass: identify entity boundaries and lengths
            entity_spans = []
            if item['annotations'] and len(item['annotations']) > 0:
                for ann in item['annotations'][0]['result']:
                    if 'value' in ann:
                        start = ann['value']['start']
                        end = ann['value']['end']
                        label = ann['value']['labels'][0]
                        entity_spans.append((start, end, label))

            # Sort spans by start position
            entity_spans.sort(key=lambda x: x[0])

            # Second pass: apply BIOES labels
            for start, end, label in entity_spans:
                # Get tokens that are part of this entity
                entity_text = text[start:end]
                entity_doc = self.nlp(entity_text)
                entity_length = len([token for token in entity_doc if not token.is_space])

                # Set labels for the entire span
                current_token_idx = 0
                for i in range(start, end):
                    if i == start or text[i-1].isspace():
                        char_labels[i] = self.get_bioes_label(i, entity_length, current_token_idx, label)
                        current_token_idx += 1
                    else:
                        char_labels[i] = char_labels[i-1]

            # Convert to token-level labels
            current_sentence = []
            for sent in doc.sents:
                for token in sent:
                    # Get the most common label for the token's characters
                    token_chars_labels = char_labels[token.idx:token.idx + len(token.text)]
                    label_counts = defaultdict(int)
                    for char_label in token_chars_labels:
                        label_counts[char_label] += 1

                    token_label = max(label_counts.items(), key=lambda x: x[1])[0]
                    current_sentence.append((token.text, token_label))

                if current_sentence:
                    all_sentences.append(current_sentence)
                    current_sentence = []

        return all_sentences

    def write_flair_file(self, sentences: List[List[Tuple[str, str]]], filename: str):
        """Write sentences in BIOES format to file."""
        with open(filename, 'w', encoding='utf-8') as f:
            for sentence in sentences:
                for token, label in sentence:
                    f.write(f'{token} {label}\n')
                f.write('\n')

    def convert_and_split(self, json_data: List[dict], train_file: str, test_file: str, test_ratio: float = 0.2):
        """Convert JSON to BIOES format and split into train/test sets."""
        all_sentences = self.convert_to_bioes_format(json_data)

        # Shuffle and split
        random.shuffle(all_sentences)
        split_idx = int(len(all_sentences) * (1 - test_ratio))

        train_sentences = all_sentences[:split_idx]
        test_sentences = all_sentences[split_idx:]

        # Write to files
        self.write_flair_file(train_sentences, train_file)
        self.write_flair_file(test_sentences, test_file)

        return len(train_sentences), len(test_sentences)

def main():
    # Load JSON data
    with open('/content/drive/MyDrive/FYP/Implementation/Resume Dataset/200_resumes_annotated.json', 'r', encoding='utf-8') as f:
        json_data = json.load(f)

    # Convert and split data
    converter = NERConverter()
    train_count, test_count = converter.convert_and_split(
        json_data,
        train_file='flair_train.txt',
        test_file='flair_test.txt',
        test_ratio=0.2
    )

    print(f'Created {train_count} training sentences and {test_count} test sentences')

main()

Created 298 training sentences and 75 test sentences


In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# Define columns for CoNLL (0: word, 1: label)
columns = {0: 'text', 1: 'ner'}

# Set data folder and file names
data_folder = './'
train_file = 'flair_train.txt'
test_file = 'flair_test.txt'

# Load the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file=train_file,
                              test_file=test_file,
                              dev_file=None)

2024-11-06 10:22:39,922 Reading data from .
2024-11-06 10:22:39,923 Train: flair_train.txt
2024-11-06 10:22:39,927 Dev: None
2024-11-06 10:22:39,928 Test: flair_test.txt
2024-11-06 10:22:41,745 No dev split found. Using 10% (i.e. 30 samples) of the train split as dev data


In [ ]:
tag_dictionary = corpus.make_label_dictionary(label_type='ner')
print("Labels:", tag_dictionary.get_items())

2024-11-06 10:22:43,986 Computing label dictionary. Progress:


0it [00:00, ?it/s]
268it [00:00, 12655.92it/s]

2024-11-06 10:22:44,035 Dictionary created for label 'ner' with 11 values: SKILL (seen 2196 times), JOB (seen 520 times), WORK (seen 445 times), COMPANY (seen 353 times), LOC (seen 221 times), DEG (seen 154 times), UNI (seen 152 times), STUDY (seen 133 times), NAME (seen 133 times), PHONE (seen 132 times), EMAIL (seen 99 times)
Labels: ['SKILL', 'JOB', 'WORK', 'COMPANY', 'LOC', 'DEG', 'UNI', 'STUDY', 'NAME', 'PHONE', 'EMAIL']


In [ ]:
from collections import Counter

def count_labels(file_path):
    with open(file_path, 'r') as file:
        labels = [line.split()[-1] for line in file if line.strip()]
    return Counter(labels)

print("Train label distribution:", count_labels('flair_train.txt'))
print("Test label distribution:", count_labels('flair_test.txt'))

Train label distribution: Counter({'O': 71084, 'S-SKILL': 1461, 'PER': 1178, 'B-SKILL': 950, 'E-SKILL': 946, 'E-JOB': 521, 'B-JOB': 511, 'E-COMPANY': 340, 'B-COMPANY': 333, 'I-JOB': 283, 'I-DEG': 268, 'I-COMPANY': 254, 'I-SKILL': 191, 'E-UNI': 170, 'B-UNI': 169, 'B-DEG': 160, 'E-DEG': 160, 'S-LOC': 147, 'B-NAME': 144, 'E-NAME': 144, 'I-UNI': 127, 'E-PHONE': 116, 'B-PHONE': 114, 'S-EMAIL': 108, 'B-LOC': 101, 'E-LOC': 101, 'I-PHONE': 82, 'S-COMPANY': 63, 'S-JOB': 62, 'S-PHONE': 23, 'I-NAME': 10, 'S-DEG': 6, 'S-UNI': 2, 'I-LOC': 1, 'B-EMAIL': 1, 'E-EMAIL': 1})
Test label distribution: Counter({'O': 18009, 'PER': 333, 'S-SKILL': 330, 'B-SKILL': 263, 'E-SKILL': 263, 'E-JOB': 138, 'B-JOB': 136, 'B-COMPANY': 89, 'E-COMPANY': 87, 'I-JOB': 74, 'I-DEG': 66, 'I-COMPANY': 61, 'S-LOC': 58, 'B-NAME': 47, 'E-NAME': 47, 'I-SKILL': 45, 'B-UNI': 45, 'E-UNI': 45, 'I-UNI': 44, 'E-DEG': 42, 'B-DEG': 41, 'B-PHONE': 41, 'E-PHONE': 41, 'S-EMAIL': 40, 'B-LOC': 27, 'E-LOC': 27, 'I-PHONE': 24, 'S-COMPANY': 16, '

In [ ]:
# create NER tagger
from flair.embeddings import WordEmbeddings, StackedEmbeddings, TransformerWordEmbeddings, FlairEmbeddings
from flair.models import SequenceTagger

# using LSTM-CRF on top of frozen embeddings
# combine flair and glove embeddings
# embeddings = StackedEmbeddings([
#                 WordEmbeddings('glove'),
#                 FlairEmbeddings('news-forward'),
#                 FlairEmbeddings('news-backward'),
#             ])
# tagger = SequenceTagger(hidden_size=256,
#                          embeddings=embeddings,
#                          tag_dictionary=tag_dictionary,
#                          tag_type='ner',
#                          use_crf=True,
#                          tag_format = 'BIOES')

# using transformer embedding
# embeddings = TransformerWordEmbeddings('bert-base-uncased',
#                                       fine_tune=True,
#                                       layers='-1',
#                                       subtoken_pooling='first')
embeddings = TransformerWordEmbeddings(
    'roberta-base',  # or 'bert-base-uncased'
    fine_tune=True,
    layers='-1,-2,-3,-4',  # Use last 4 layers
    subtoken_pooling='first',
    allow_long_sentences=True
)
tagger = SequenceTagger(hidden_size=256,
                         embeddings=embeddings,
                         tag_dictionary=tag_dictionary,
                         tag_type='ner',
                         use_crf=False,
                         use_rnn=False,
                         reproject_embeddings=False,
                         tag_format = 'BIOES')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


2024-11-06 10:59:16,123 SequenceTagger predicts: Dictionary with 45 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-LOC, B-LOC, E-LOC, I-LOC, S-DEG, B-DEG, E-DEG, I-DEG, S-UNI, B-UNI, E-UNI, I-UNI, S-STUDY, B-STUDY, E-STUDY, I-STUDY, S-NAME, B-NAME, E-NAME, I-NAME, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL


In [ ]:
# train flair ner model
from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric

trainer = ModelTrainer(tagger, corpus)

trainer.train(
    base_path='flair_output/',
    learning_rate=0.01,
    mini_batch_size=4,
    max_epochs=50,
    train_with_dev=False
)
!cp -r ./flair_output /content/drive/MyDrive/FYP/Implementation/

2024-11-06 10:23:02,204 ----------------------------------------------------------------------------------------------------
2024-11-06 10:23:02,207 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50266, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768

/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-11-06 10:23:05,898 epoch 1 - iter 6/67 - loss 2.77562790 - time (sec): 3.66 - samples/sec: 1991.31 - lr: 0.010000 - momentum: 0.000000
2024-11-06 10:23:07,877 epoch 1 - iter 12/67 - loss 2.17279687 - time (sec): 5.64 - samples/sec: 2162.44 - lr: 0.010000 - momentum: 0.000000
2024-11-06 10:23:10,314 epoch 1 - iter 18/67 - loss 1.73979483 - time (sec): 8.08 - samples/sec: 2274.98 - lr: 0.010000 - momentum: 0.000000
2024-11-06 10:23:13,007 epoch 1 - iter 24/67 - loss 1.49670987 - time (sec): 10.77 - samples/sec: 2281.43 - lr: 0.010000 - momentum: 0.000000
2024-11-06 10:23:15,268 epoch 1 - iter 30/67 - loss 1.40065300 - time (sec): 13.03 - samples/sec: 2273.90 - lr: 0.010000 - momentum: 0.000000
2024-11-06 10:23:17,923 epoch 1 - iter 36/67 - loss 1.34974097 - time (sec): 15.69 - samples/sec: 2286.83 - lr: 0.010000 - momentum: 0.000000
2024-11-06 10:23:20,103 epoch 1 - iter 42/67 - loss 1.27069429 - time (sec): 17.87 - samples/sec: 2298.39 - lr: 0.010000 - momentum: 0.000000
2024-11-06

100%|██████████| 1/1 [00:02<00:00,  2.93s/it]

2024-11-06 10:23:34,284 DEV : loss 0.4303983449935913 - f1-score (micro avg)  0.0
2024-11-06 10:23:34,304  - 0 epochs without improvement
2024-11-06 10:23:34,305 ----------------------------------------------------------------------------------------------------


...

100%|██████████| 1/1 [00:08<00:00,  8.66s/it]

2024-11-06 10:56:19,904 DEV : loss 0.3591618239879608 - f1-score (micro avg)  0.0
2024-11-06 10:56:19,925  - 0 epochs without improvement


2024-11-06 10:56:26,085 ----------------------------------------------------------------------------------------------------
2024-11-06 10:56:26,089 Testing using last state of model ...


100%|██████████| 2/2 [00:05<00:00,  2.74s/it]

2024-11-06 10:56:31,631 
Results:
- F-score (micro) 0.563
- F-score (macro) 0.7187
- Accuracy 0.3962

By class:
              precision    recall  f1-score   support

       SKILL     0.3408    0.3862    0.3621       593
         JOB     0.5730    0.7020    0.6310       151
        WORK     0.8188    0.9457    0.8777       129
     COMPANY     0.5172    0.5714    0.5430       105
         LOC     0.5701    0.7176    0.6354        85
         DEG     0.5862    0.7907    0.6733        43
         UNI     0.5660    0.6667    0.6122        45
        NAME     0.9388    0.9787    0.9583        47
       PHONE     1.0000    0.9787    0.9892        47
       EMAIL     0.8222    0.9250    0.8706        40
       STUDY     0.8286    0.6905    0.7532        42

   micro avg     0.5281    0.6029    0.5630      1327
   macro avg     0.6874    0.7594    0.7187      1327
weighted avg     0.5324    0.6029    0.5642      1327

2024-11-06 10:56:31,633 ---------------------------------------------------

In [ ]:
# evaluate model
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

# Load the trained model
model = SequenceTagger.load('/content/drive/MyDrive/FYP/Implementation/flair_output/final-model.pt')

# Evaluate the model on the test set
result = model.evaluate(corpus.test, gold_label_type='ner', mini_batch_size=32)

# Print the results
# print("Evaluation Loss:", eval_loss)
print(result.detailed_results)  # print the precision, recall, and F1-score per entity type

2024-11-06 10:57:38,582 SequenceTagger predicts: Dictionary with 45 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-LOC, B-LOC, E-LOC, I-LOC, S-DEG, B-DEG, E-DEG, I-DEG, S-UNI, B-UNI, E-UNI, I-UNI, S-STUDY, B-STUDY, E-STUDY, I-STUDY, S-NAME, B-NAME, E-NAME, I-NAME, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL


100%|██████████| 3/3 [00:03<00:00,  1.29s/it]


Results:
- F-score (micro) 0.563
- F-score (macro) 0.7187
- Accuracy 0.3962

By class:
              precision    recall  f1-score   support

       SKILL     0.3408    0.3862    0.3621       593
         JOB     0.5730    0.7020    0.6310       151
        WORK     0.8188    0.9457    0.8777       129
     COMPANY     0.5172    0.5714    0.5430       105
         LOC     0.5701    0.7176    0.6354        85
         DEG     0.5862    0.7907    0.6733        43
         UNI     0.5660    0.6667    0.6122        45
        NAME     0.9388    0.9787    0.9583        47
       PHONE     1.0000    0.9787    0.9892        47
       EMAIL     0.8222    0.9250    0.8706        40
       STUDY     0.8286    0.6905    0.7532        42

   micro avg     0.5281    0.6029    0.5630      1327
   macro avg     0.6874    0.7594    0.7187      1327
weighted avg     0.5324    0.6029    0.5642      1327



In [ ]:
from flair.models import SequenceTagger
from flair.data import Sentence
import spacy
from spacy import displacy

# Load your trained Flair NER model
tagger = SequenceTagger.load('/content/drive/MyDrive/FYP/Implementation/flair_output/best-model.pt')

resume_text = '''
John Doe lives at 1234 Elm Street in Los Angeles, CA 90001. He can be reached at +1 (555) 123-4567 or via email at john.doe@example.com. John is a results-driven software engineer with over 5 years of experience in web development and cloud infrastructure, with strong knowledge of JavaScript, Python, and cloud technologies like AWS and Azure. Currently, he works as a Software Engineer at Google LLC in San Francisco, CA, where he has been employed since August 2019. In this role, he has developed scalable web applications using JavaScript, Node.js, and React, deployed and maintained cloud infrastructure on AWS, reducing downtime by 20%, and led a team of 4 engineers to enhance backend performance by 30%. Previously, he worked as a Junior Developer at Tech Innovators Inc. in Austin, TX, from July 2017 to July 2019, where he created RESTful APIs using Python and Flask, collaborated with front-end developers to build and deploy user-facing applications, and wrote unit and integration tests, improving code coverage by 15%.

John holds a Master of Science in Computer Science from the University of California, Berkeley, with a graduation date of May 2017, and a Bachelor of Science in Information Technology from the University of Texas at Austin, graduated in May 2015. His skillset includes proficiency in programming languages like Python, JavaScript, and Java; frameworks such as React, Flask, and Django; cloud platforms including AWS, Google Cloud, and Azure; as well as other tools like Git, Docker, Kubernetes, and SQL. He is certified as an AWS Certified Solutions Architect – Associate, earned in 2020, and as a Google Professional Cloud Architect, earned in 2021'
'''

# Step 1: Predict entities using Flair
sentence = Sentence(resume_text)
tagger.predict(sentence)

# Step 2: Convert Flair predictions to spaCy doc format
# Initialize a blank spaCy NLP pipeline
nlp = spacy.blank("en")
doc = nlp(resume_text)

# Extract entities from Flair prediction and convert to spaCy format
ents = []
for entity in sentence.get_spans('ner'):
    start, end = entity.start_position, entity.end_position
    label = entity.tag
    span = doc.char_span(start, end, label=label)
    if span is not None:
        ents.append(span)

# Set the entities in the spaCy doc
doc.ents = ents

# Step 3: Visualize using displacy
# Display in Jupyter or a web page
displacy.render(doc, style="ent", jupyter=True)


2024-11-06 10:57:55,781 SequenceTagger predicts: Dictionary with 47 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-LOC, B-LOC, E-LOC, I-LOC, S-UNI, B-UNI, E-UNI, I-UNI, S-DEG, B-DEG, E-DEG, I-DEG, S-NAME, B-NAME, E-NAME, I-NAME, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-STUDY, B-STUDY, E-STUDY, I-STUDY, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL, <START>, <STOP>


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
from flair.models import SequenceTagger
from flair.data import Sentence

# Load the pretrained NER model
tagger = SequenceTagger.load("/content/drive/MyDrive/FYP/Implementation/flair_output/best-model.pt")
# Example text
text = "Apple is looking at buying U.K. startup for $1 billion."

# Create a Sentence object
sentence = Sentence(resume_text)

# Predict entities
tagger.predict(sentence)

# Print the detected entities
for entity in sentence.get_spans("ner"):
    print(f"Entity: {entity.text}, Type: {entity.get_label('ner').value}, Confidence: {entity.score}")


2024-11-06 10:58:17,528 SequenceTagger predicts: Dictionary with 47 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-LOC, B-LOC, E-LOC, I-LOC, S-UNI, B-UNI, E-UNI, I-UNI, S-DEG, B-DEG, E-DEG, I-DEG, S-NAME, B-NAME, E-NAME, I-NAME, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-STUDY, B-STUDY, E-STUDY, I-STUDY, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL, <START>, <STOP>
Entity: Doe lives, Type: NAME, Confidence: 0.16702505946159363
Entity: 1234 Elm, Type: NAME, Confidence: 0.11599351465702057
Entity: Los, Type: DEG, Confidence: 0.18544061481952667
Entity: 90001, Type: DEG, Confidence: 0.14748063683509827
Entity: 555, Type: DEG, Confidence: 0.11017131805419922
Entity: 123-4567, Type: DEG, Confidence: 0.11569119244813919
Entity: via, Type: DEG, Confidence: 0.20629706978797913
Entity: at, Type: EMAIL, Confidence: 0.19365909695625305
Entity: john.doe, Type: JOB, Confidence: 0.10587572306394577
Entity: @, Ty